# 1) From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only. (20 points)

In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Assignment2").getOrCreate()
sc = spark.sparkContext

In [2]:
from xml.etree import ElementTree
import re

tree = ElementTree.parse("users.xml")
item = tree.getroot()
users=[]

for i in item:
    users.append(i.attrib)
    
usersRDD = sc.parallelize(users)

locationLines = usersRDD.filter(lambda x: "Location" in x)
georgiaUsersRDD = locationLines.filter(lambda x: "Georgia" in x['Location'])
displayNameMap = georgiaUsersRDD.map(lambda x: x['DisplayName'])

print("User's Display Names whose Location is Georgia::")
print(displayNameMap.collect())

User's Display Names whose Location is Georgia::
['Ayush', 'vkb', 'Giorgi Gzirishvili', 'Alex Azazel', 'Dan Anton', 'Manish Ranjan', 'Dato Janez', 'Neuromeda', 'Abesalomi Gogatishvili', 'Jenna Kwon', 'Tedo G.', 'mlibre', 'Jeff', 'Jesse Scherer', 'Melinda Weathers', 'N.C.W.', 'G.T.', 'Dato Gogolashvili', 'David Jashi', 'Ilia Chigogidze', 'Tae-Sung Shin', 'Clark Lin', 'Gio Nari', 'alerera', 'Max Boonbandansook']


# 2) Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017. (30 points). Output this to the screen.

In [3]:
creationDateRDD = usersRDD.filter(lambda x: "CreationDate" in x)
users2017RDD = creationDateRDD.filter(lambda x: "2017" in x['CreationDate'])
print("Count of all users who joined (Creation Date) in 2017 =" , users2017RDD.count())
        

Count of all users who joined (Creation Date) in 2017 = 14239


# 3) Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen.

In [4]:
from xml.etree import ElementTree
import re

tree = ElementTree.parse("PostHistory.xml")
item = tree.getroot()
postHistory=[]

for i in item:
    postHistory.append(i.attrib)
    
postHistoryRDD = sc.parallelize(postHistory)

textLines = postHistoryRDD.filter(lambda x: "Text" in x)
sparkScalaLines = textLines.filter(lambda x: "spark" in x['Text'].lower() and "scala" in x['Text'].lower() )
          
print("Count of posts featuring the words 'Spark' and 'Scala' =" , sparkScalaLines.count())

Count of posts featuring the words 'Spark' and 'Scala' = 211


# 4) Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID (30 points). You get 15 bonus points if you get the actual DisplayName of the user.

In [5]:
textLines = postHistoryRDD.filter(lambda x: "Text" in x)
userIdLinesWithText = textLines.filter(lambda x: "UserId" in x)
userIdTextRDD = userIdLinesWithText.map(lambda x: (x['UserId'], len(x['Text'].split(" "))))
idTextCount = userIdTextRDD.reduceByKey(lambda a, b: a + b)

userDisplayName = usersRDD.map(lambda x: (x['Id'], x['DisplayName']))

display = idTextCount.join(userDisplayName)
print(display.collect())

[('24', (1294, 'buruzaemon')), ('113', (903, 'vefthym')), ('180', (54, 'Oleksi')), ('283', (243, 'Alireza')), ('478', (331, 'Stanpol')), ('508', (43, 'azza-bazoo')), ('553', (355, 'lsdr')), ('50', (6, 'Robert Cartaino')), ('957', (75, 'user3663635')), ('816', (7510, 'Alexey Grigorev')), ('1352', (786, 'Stephane Rolland')), ('2458', (496, 'johnhidley')), ('2653', (89, 'aidankmcl')), ('2928', (631, 'universalkernel')), ('2948', (33, 'Noor')), ('2958', (374, 'Maxim Galushka')), ('3030', (119, 'user1913171')), ('3153', (86, 'Ted Dunning')), ('3259', (26, 'Harjeet Kumar')), ('3598', (41, 'Rakhi Ssdd')), ('4676', (348, 'PlagTag')), ('3281', (5653, 'Guforu')), ('4847', (16, 'iboboboru')), ('4859', (122, 'aa51')), ('4929', (20, 'Jerome')), ('5004', (209, 'Nick Lashinsky')), ('3377', (676, 'AndriusZ')), ('5023', (147, 'k6adams')), ('5083', (112, 'Oleg Roderick')), ('5153', (959, 'goangit')), ('5165', (310, 'JC R')), ('5227', (115, 'ccanduc')), ('5290', (880, 'Stumbler')), ('6606', (446, 'flow')

# 5) GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower.

In [8]:
from xml.etree import ElementTree
import re

tree = ElementTree.parse("Comments.xml")
item = tree.getroot()
comments=[]

for i in item:
    comments.append(i.attrib)
    
commentsRDD = sc.parallelize(comments)

commentsWithUserIdLines = commentsRDD.filter(lambda x: "UserId" in x)
userIdFromCommentsRDD = commentsWithUserIdLines.map(lambda x: (x['UserId'], 1))
userIdCommentsCount = userIdFromCommentsRDD.reduceByKey(lambda a, b: a + b)

scoreFromCommentsLines = commentsWithUserIdLines.filter(lambda x: "Score" in x)
scoreUserIdComments = scoreFromCommentsLines.map(lambda x: (x['UserId'], int(x['Score'])))
scoreUserIDCount = scoreUserIdComments.reduceByKey(lambda a, b: a + b)

postsNumberLines = postHistoryRDD.filter(lambda x: "UserId" in x)
userIdFromPostsRDD = postsNumberLines.map(lambda x: (x['UserId'], 1))
userIdPostsCount = userIdFromPostsRDD.reduceByKey(lambda a, b: a + b)

userIdLines = usersRDD.filter(lambda x: "Id" in x)
displayNameMap = userIdLines.map(lambda x: (x['Id'], x['DisplayName']))


In [9]:
x = displayNameMap.join(userIdCommentsCount)
y = x.join(scoreUserIDCount)
z = y.join(userIdPostsCount)


singleArray = z.map(lambda x: (x[0], x[1][1], x[1][0][0][0], x[1][0][0][1], x[1][0][1]))
print(singleArray.collect())

[('24', 11, 'buruzaemon', 4, 10), ('102', 4, 'Bryan', 1, 0), ('113', 10, 'vefthym', 4, 6), ('816', 38, 'Alexey Grigorev', 17, 7), ('2458', 5, 'johnhidley', 1, 0), ('5290', 13, 'Stumbler', 1, 0), ('6660', 3, 'cakesofwrath', 1, 0), ('8065', 8, 'Leon palafox', 7, 0), ('8074', 3, 'Nart Barileva', 1, 0), ('8729', 16, 'Jaishree Rout', 2, 0), ('9114', 5, 'user3780968', 2, 0), ('9270', 3, 'firelynx', 1, 0), ('9838', 3, 'David', 1, 1), ('14560', 26, 'Philip C. ', 27, 8), ('17461', 1, 'Roberto Ferrer', 2, 0), ('17538', 20, 'E_net4', 36, 7), ('18271', 3, 'Dider', 1, 0), ('20759', 8, 'Igor OA', 4, 2), ('23478', 2, 'layser', 1, 0), ('25183', 9, 'utengr', 2, 2), ('29083', 4, 'Adam Sypniewski', 5, 3), ('29451', 13, 'Dan Jarratt', 6, 0), ('30039', 3, 'Futuregeek', 1, 0), ('30107', 42, 'Rahul Aedula', 34, 5), ('30121', 8, 'Marcin', 1, 0), ('30781', 1, 'Vineet Kothari', 1, 0), ('33907', 8, 'Patthebug', 2, 0), ('35902', 9, 'hongsy', 1, 0), ('37113', 5, 'jtlz2', 2, 0), ('38630', 3, 'Mathias Mamsch', 1, 0)

In [15]:
import numpy as np

np.savetxt("Question5Output.txt", np.array(singleArray.collect()), fmt="%s", encoding='utf-8')